In [1]:
import pickle
import tensorflow as tf
import numpy as np
import tf_util
import gym
import load_policy

from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape
from keras.utils import np_utils
from sklearn.utils import shuffle

%load_ext autoreload
%autoreload 2


/home/tiger/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
def load_task_data(filename):
    with open(filename, 'rb') as f:
        task_data = pickle.loads(f.read())
    return task_data

In [11]:
num_rollouts = 20
expert_name = "Hopper-v1"
#expert_name = "Ant-v1"
#expert_name = "HalfCheetah-v1"
#expert_name = "Humanoid-v1"
#expert_name = "Reacher-v1"
#expert_name = "Walker2d-v1"


In [13]:
#import argparse
"""
parser = argparse.ArgumentParser()
parser.add_argument('expert_policy_file', type=str)
parser.add_argument('envname', type=str)
parser.add_argument('--render', action='store_true')
parser.add_argument("--max_timesteps", type=int)
parser.add_argument('--num_rollouts', type=int, default=20,
                    help='Number of expert roll outs')
args = parser.parse_args()
"""



print('loading and building expert policy')
expert_policy_file = "./experts/" + expert_name + ".pkl"
policy_fn = load_policy.load_policy(expert_policy_file)
print('loaded and built')



with tf.Session():
    tf_util.initialize()

    import gym
    env = gym.make(expert_name)
    max_steps = env.spec.timestep_limit

    returns = []
    observations = []
    actions = []
    for i in range(num_rollouts):
        print('iter', i)
        obs = env.reset()
        
        print("obs.shape:", obs)
        done = False
        totalr = 0.
        steps = 0
        while not done:
            action = policy_fn(obs[None,:])
            observations.append(obs)
            actions.append(action)
            obs, r, done, _ = env.step(action)
            totalr += r
            steps += 1
            #if args.render:
            env.render()
            if steps % 100 == 0: print("%i/%i"%(steps, max_steps))
            if steps >= max_steps:
                break
        returns.append(totalr)

    print('returns', returns)
    print('mean return', np.mean(returns))
    print('std of return', np.std(returns))

    expert_data = {'observations': np.array(observations),
                   'actions': np.array(actions)}
    
    output_file_name = 'data/' + expert_name + '_' + str(num_rollouts) + '_data.pkl'
    with open(output_file_name, 'wb') as f:
        pickle.dump(expert_data, f)

[2018-04-13 10:21:13,772] Making new env: Hopper-v1


loading and building expert policy
obs (1, 11) (1, 11)
loaded and built
iter 0
obs.shape: [  1.24556417e+00   3.04872868e-03  -4.32852109e-03  -2.69590324e-03
  -1.32229762e-04  -3.52707980e-03  -8.48929954e-04   2.73394667e-03
   3.96865475e-03   3.07426810e-03   2.23377431e-03]
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
1000/1000
iter 1
obs.shape: [  1.24839728e+00   1.51433512e-03  -1.97075616e-03  -1.17631985e-03
  -4.04536503e-03   4.64813848e-03   4.51193253e-03   3.50786108e-03
  -8.59176954e-04  -2.37771297e-03   2.37430879e-03]
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
1000/1000
iter 2
obs.shape: [  1.25349291e+00  -8.67386145e-04   1.26907689e-03   4.97756671e-03
   4.98735347e-03   4.60919367e-04   1.98723506e-03  -4.24295464e-03
   3.79993192e-03   2.49235089e-03   3.24791804e-05]
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
1000/1000
iter 3
obs.shape: [  1.24651

In [6]:
with open(output_file_name, 'wb') as f:
    pickle.dump(expert_data, f)

In [8]:
print (max_steps)

1000
